In [25]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import constants
from numba import njit
import plotly.graph_objects as go

def initial_state(N):
    '''Generates a random spin configuration for initial condition.'''
    state = np.random.choice([-1, 1], size=(N, N, N))
    return state

@njit
def mc_move(config, beta):
    '''Monte Carlo move using Metropolis algorithm '''
    for i in range(N):
        for j in range(N):
            for k in range(N):
                a = np.random.randint(0, N)
                b = np.random.randint(0, N)
                c = np.random.randint(0, N)
                s =  config[a, b, c]
                nb = config[(a+1)%N,b,c] + config[a,(b+1)%N,c] + config[a,b,(c+1)%N] + config[(a-1)%N,b,c] + config[a,(b-1)%N,c] + config[a,b,(c-1)%N]
                cost = 2*s*nb
                if cost < 0:
                    s *= -1
                elif np.random.rand() < np.exp(-cost*beta):
                    s *= -1
                config[a, b, c] = s
    return config

@njit
def calculate_energy(config):
    '''Energy of a given configuration'''
    energy = 0
    for i in range(len(config)):
        for j in range(len(config)):
            for k in range(len(config)):
                S = config[i,j,k]
                nb = config[(i+1)%N, j, k] + config[i,(j+1)%N, k] + config[i, j, (k+1)%N] + config[(i-1)%N, j, k] + config[i,(j-1)%N, k] + config[i, j, (k-1)%N]
                energy += -nb*S
    return energy/2.  # To compensate for double counting

def simulate(N=10, temp=2.0, num_steps=1000):
    ''' Main function to execute the MC simulation '''
    config = initial_state(N)
    beta = 1.0/(temp)
    
    for i in range(num_steps):
        mc_move(config, beta)
    
    energy = calculate_energy(config)
    return config, energy

# def visualize_state(config):
#     '''Visualizes the spin configuration in 3D.'''
#     fig = plt.figure(figsize=(10, 8))
#     ax = fig.add_subplot(111, projection='3d')
    
#     # Plot spins: +1 spins will be blue, -1 spins will be red
#     for i in range(config.shape[0]):
#         for j in range(config.shape[1]):
#             for k in range(config.shape[2]):
#                 if config[i, j, k] > 0:
#                     ax.scatter(i, j, k, color='blue', marker='o')
#                 else:
#                     ax.scatter(i, j, k, color='red', marker='x')
    
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
#     plt.title('3D Ising Model Spin Configuration')
#     plt.show()

def visualize_state_plotly(config):
    '''Visualizes the spin configuration in 3D using Plotly.'''
    up_spins_x, up_spins_y, up_spins_z = np.where(config == 1)
    down_spins_x, down_spins_y, down_spins_z = np.where(config == -1)
    
    trace_up = go.Scatter3d(
        x=up_spins_x, y=up_spins_y, z=up_spins_z,
        mode='markers',
        marker=dict(
            size=5,
            color='blue',  # Up spins are blue
            opacity=0.1
        ),
        name='Up Spins'
    )
    
    trace_down = go.Scatter3d(
        x=down_spins_x, y=down_spins_y, z=down_spins_z,
        mode='markers',
        marker=dict(
            size=0,
            color='white',  # Down spins are red
            opacity=0.0
        ),
        name='Down Spins'
    )
    
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )
    
    fig = go.Figure(data=[trace_up], layout=layout)
    fig.show()

In [27]:
# Parameters
N = 50  # Dimension of the lattice, N x N x N
temp = 2  # Temperature in units
num_steps = 1000  # Number of MC steps

config, energy = simulate(N, temp, num_steps)
print(f"Energy: {energy}")
visualize_state_plotly(config)

Energy: -370892.0
